
# Column Transformer with Mixed Types


This example illustrates how to apply different preprocessing and feature
extraction pipelines to different subsets of features, using
`ColumnTransformer`. This is particularly handy for the
case of datasets that contain heterogeneous data types, since we may want to
scale the numeric features and one-hot encode the categorical ones.

In this example, the numeric data is standard-scaled after mean-imputation,
while the categorical data is one-hot encoded after imputing missing values
with a new category (``'missing'``).

In addition, we show two different ways to dispatch the columns to the
particular pre-processor: by column names and by column data types.

Finally, the preprocessing pipeline is integrated in a full prediction pipeline
using `Pipeline`, together with a simple classification
model.


In [2]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

#### Step 1: Read the data set `Titanic.csv`
Print the column names. Drop 'boat','body','cabin','home.dest','name','ticket','sibsp','parch'

In [57]:
X_all = pd.read_csv('Titanic.csv')

In [58]:
X_all.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [6]:
X_all.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [59]:
Data = X_all.drop(['boat','body','home.dest','cabin','name','ticket','sibsp','parch'],axis=1)

#### Step 2: Calculate duplicates
- Report the size of the dataset
- Report if there are any duplicates using `duplicated()` method of pandas dataframe
- List all duplicate rows
- Drop all duplicate rows using `dop_duplicates` method of pandas dataframe
- Report the new shape of the dataset

In [9]:
Data.shape

(1309, 6)

In [11]:
dups = Data.duplicated()

In [12]:
dups.sum()

215

In [14]:
Data.drop_duplicates(inplace=True)

#### Step 3: Explore the dataset for missing values
- Assign `survived' as target variable `y` and the rest as `X`
- Print info about `X` to see if there are any null values and the type of the features
- Check if all columns have only alpha numeric characters. To do that you need to use `isalnum()` method of Python which works on strings. Before using this method. we need to change all features into string using `astype('str')`
- Replace all `?` characters with `np.nan`
- Now, turn 'age','fare' into numeric again using `apply(pd.to_numeric) `
- Print the first five samples of `X`
- Summarize the number of unique values in each column using `nunique()`

In [60]:
X = Data.drop('survived', axis=1)
y = Data.survived

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1094 non-null   object 
 1   sex       1094 non-null   object 
 2   age       966 non-null    float64
 3   fare      1094 non-null   object 
 4   embarked  1094 non-null   object 
dtypes: float64(1), object(4)
memory usage: 51.3+ KB


In [19]:
# isalnum: does it only include alphabetic and number?
'123'.isalnum()


True

In [20]:
'?J'.isalnum()

False

In [21]:
X.columns

Index(['pclass', 'sex', 'age', 'fare', 'embarked'], dtype='object')

In [22]:
cols = list(X.columns)
cols

['pclass', 'sex', 'age', 'fare', 'embarked']

In [53]:
X[cols] = X[cols].astype(str)

In [52]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [27]:
X.loc[:,'age'].str.isalnum()

0        True
1       False
2        True
3        True
4        True
        ...  
1301    False
1304    False
1306    False
1307     True
1308     True
Name: age, Length: 1094, dtype: bool

In [28]:
X.loc[:,'age']

0           29
1       0.9167
2            2
3           30
4           25
         ...  
1301      45.5
1304      14.5
1306      26.5
1307        27
1308        29
Name: age, Length: 1094, dtype: object

In [32]:
for i in range(len(cols)):
    print((~X.iloc[:,i].str.isalnum()).sum())
    # ~ will make True to false, false into true

0
0
172
898
2


In [40]:
X[(~X.iloc[:,4].str.isalnum())].iloc[:,4]

168    ?
284    ?
Name: embarked, dtype: object

In [41]:
np.unique(X.embarked)

array(['?', 'C', 'Q', 'S'], dtype=object)

In [51]:
X.age = X.age.replace({'?':'NaN'})

In [45]:
X[X.iloc[:,2]=='NaN']

,pclass,sex,age,fare,embarked
15,1,male,NaN,25.925,S
37,1,male,NaN,26.55,S
40,1,male,NaN,39.6,C
46,1,male,NaN,31,S
59,1,female,NaN,27.7208,C
...,...,...,...,...,...
1198,3,female,NaN,7.7792,Q
1214,3,male,NaN,8.6625,S
1241,3,male,NaN,6.4375,C
1268,3,male,NaN,9.5,S


In [50]:
X.age = X.age.astype('float')

ValueError: could not convert string to float: '?'

In [47]:
X[pd.isnull(X.age)]

,pclass,sex,age,fare,embarked
15,1,male,NaN,25.925,S
37,1,male,NaN,26.55,S
40,1,male,NaN,39.6,C
46,1,male,NaN,31,S
59,1,female,NaN,27.7208,C
...,...,...,...,...,...
1198,3,female,NaN,7.7792,Q
1214,3,male,NaN,8.6625,S
1241,3,male,NaN,6.4375,C
1268,3,male,NaN,9.5,S


In [61]:
X = X.replace({'?':np.nan})

In [62]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1309 non-null   int64 
 1   sex       1309 non-null   object
 2   age       1046 non-null   object
 3   fare      1308 non-null   object
 4   embarked  1307 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [65]:
X[['age','fare']] = X[['age','fare']].apply(pd.to_numeric)

In [66]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   sex       1309 non-null   object 
 2   age       1046 non-null   float64
 3   fare      1308 non-null   float64
 4   embarked  1307 non-null   object 
 5   flare     1308 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 61.5+ KB


In [67]:
X = X.drop('flare', axis=1)

In [68]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   sex       1309 non-null   object 
 2   age       1046 non-null   float64
 3   fare      1308 non-null   float64
 4   embarked  1307 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 51.3+ KB


In [69]:
X[X['embarked'].isnull()]

,pclass,sex,age,fare,embarked
168,1,female,38.0,80.0,NaN
284,1,female,62.0,80.0,NaN


In [70]:
X['embarked'] = X['embarked'].astype('str')

In [71]:
X[X['embarked'].isnull()]

,pclass,sex,age,fare,embarked


In [72]:
X.loc[168,:]

pclass           1
sex         female
age           38.0
fare          80.0
embarked       nan
Name: 168, dtype: object

#### Step 4: Use `ColumnTransformer` by selecting column by names

##############################################################################

 We will train our classifier with the following features:

 Numeric Features:

 * ``age``: float;
 * ``fare``: float.

 Categorical Features:

 * ``embarked``: categories encoded as strings ``{'C', 'S', 'Q'}``;
 * ``sex``: categories encoded as strings ``{'female', 'male'}``;
 * ``pclass``: ordinal integers ``{1, 2, 3}``.

 Create the preprocessing pipelines for both numeric and categorical data.
 - For numeric features, create a pipeline with using imputer with median strategy and standard scaler
 - For categorical features, use imputer with most frequent and onehot encoder

 Use columntransformer to transform the features

 Append the preprocessor to the KNeighborsClassifier


In [73]:
numeric_features = ['age', 'fare']

In [74]:
numeric_transformer = Pipeline(steps=[
    ('imputer1', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [75]:
categorical_features = ['embarked', 'sex', 'pclass']

In [76]:
categorical_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))])

In [77]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)])

In [78]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer1',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare']),
                                ('cat',
                                 Pipeline(steps=[('imputer2',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'sex', 'pclass'])])

In [79]:
knn = KNeighborsClassifier(n_neighbors=5)

In [80]:
knnpipe = Pipeline(steps=[('prep', preprocessor),
                ('knn', knn)])

In [81]:
knnpipe

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer1',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer2',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [83]:
knnpipe.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer1',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer2',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

In [84]:
knnpipe.score(X_train, y_train)

0.8705402650356778

In [85]:
knnpipe.score(X_test, y_test)

0.7621951219512195

In [86]:
X_test.iloc[0,:]

pclass          3
sex          male
age          25.0
fare        7.775
embarked        S
Name: 1129, dtype: object

In [87]:
X_new = [2,'female',45,20,'S']

In [91]:
knnpipe.predict([X_new])

ValueError: Found array with dim 3. None expected <= 2.

Using the prediction pipeline in a grid search

##############################################################################

 Grid search can also be performed on the different preprocessing steps
 defined in the `ColumnTransformer` object, together with the classifier's
 hyperparameters as part of the `Pipeline`.
 We will search for both the imputer strategy of the numeric preprocessing
 and the number of neighbor parameter of the kneighbors classifier using
 `GridSearchCV`.



In [99]:
param = knnpipe.steps[1][0] + '__n_neighbors'
param

'knn__n_neighbors'

In [100]:
param_grid = {param:np.arange(1,15,2)}

In [101]:
GridSearchCV(knnpipe, param_grid=param_grid, cv=5)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer1',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer2',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['embarked',
                                                                          'sex',
                                                                          'pclass'])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13])})

In [103]:
cat_features = X.dtypes == 'object'

pclass      False
sex          True
age         False
fare        False
embarked     True
dtype: bool

In [ ]:
num_features = ~cat_features

In [ ]:
ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features)])